In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd6

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-10 01:33:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  3.91MB/s    in 0.2s    

2023-02-10 01:33:14 (3.91 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
Amazon_review_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
Amazon_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
total_votes_df=Amazon_review_df.filter("total_votes>= 20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [ ]:
helpful_votes_df=total_votes_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [ ]:
#count helpful_vote_df data
print(helpful_votes_df.count())

65581


In [ ]:
#create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
review_Y_df=helpful_votes_df.filter(helpful_votes_df["vine"]=='Y')
review_Y_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48852155|R1MAOLI5FJHAFM|B013X0V11K|     610966690|BLU Studio 7.0 ll...|        Wireless|          4|          249|        261|   Y|                N|More than just a ...|Ever since the ve...| 2015-08-31|
|         US|   11556116| R9PYAUDIBJVEC|B013X0V4VM|     672788343|BLU Studio C Supe...|        Wireless|          4|    

In [ ]:
#create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
review_N_df=helpful_votes_df.filter(helpful_votes_df["vine"]=='N')
review_N_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|    

In [ ]:
from pyspark.sql.functions import count
#total no of reviews
total_reviews_y=review_Y_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % total_reviews_y)

total_reviews_N= review_N_df.count()
print("Total Number of Reviews UNPAID and 'helpful': %f" % total_reviews_N)

Total Number of Reviews PAID and 'helpful': 613.000000
Total Number of Reviews UNPAID and 'helpful': 64968.000000


In [ ]:
#5-start review for paid 
review_five_star_Y = review_Y_df.filter(review_Y_df["star_rating"]=='5')
print("5-star review data on PAID Reviews: %f" % review_five_star_Y.count())

#5-start review for Unpaid 
review_five_star_N = review_N_df.filter(review_N_df["star_rating"]=='5')
print("5-star review data on UNPAID Reviews: %f" % review_five_star_N.count())


5-star review data on PAID Reviews: 222.000000
5-star review data on UNPAID Reviews: 30543.000000


In [ ]:
#percentage of 5-star reviews for Paid
Paid_Fstar_review_percentage = (review_five_star_Y.count()/total_reviews_y)
print("Percent of 5-Star Reviews from PAID,: %f" % Paid_Fstar_review_percentage )
#percentage of 5-star reviews for UNPAID
UnPaid_Fstar_review_percentage = (review_five_star_N.count()/total_reviews_N)
print("Percent of 5-Star Reviews from UNPAID,: %f" % UnPaid_Fstar_review_percentage )

Percent of 5-Star Reviews from PAID,: 0.362153
Percent of 5-Star Reviews from UNPAID,: 0.470124
